# Grado en Ingeniería de Tecnologías y Servicios de Telecomunicación / Grado en Ingeniería Informática
### **Asignatura**: Tratamiento de Señales Visuales/Tratamiento de Señales Multimedia I
### Práctica 4: Reconocimiento de escenas con Deep Learning

---

Autor: Juan C. SanMiguel (juancarlos.sanmiguel@uam.es), Universidad Autónoma de Madrid


# Creación de la red neuronal convolucional


En este script interativo de python aprenderá a definir redes neuronales convolucionales utilizando la API de alto nivel [Keras](https://www.tensorflow.org/api_docs/python/tf/keras) disponible para Tensorflow

Tiempo estimado para completar este script: 15 minutos

# 1. Preparación del entorno de trabajo

A continuación tiene un conjunto de instrucciones para establecer el entorno de trabajo. Verifique que la versión de Python es > 3.6 y la de Tensorflow es 2.3.0

In [1]:
#%%capture
#%tensorflow_version 2.x
!pip install tensorflow==2.3.0
import tensorflow as tf
!python --version     # mostrar version de python
print('Tensorflow ' + tf.__version__) # mostrar version tensorflow

     |████████████████████████████████| 320.4 MB 47 kB/s 
     |████████████████████████████████| 20.1 MB 31.7 MB/s 
     |████████████████████████████████| 2.9 MB 36.1 MB/s 
     |████████████████████████████████| 459 kB 31.6 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.0
    Uninstalling tensorflow-2.7.0:


Python 3.7.12
Tensorflow 2.3.0


# 1.Definir capas de una red (layers)


En esta parte vamos a describir los elementos básicos para definir una red neuronal de tipo *feed-forward*. Este tipo de redes toman una serie de datos de entrada (*input*), éstos son procesados por una serie de capas (*layers*) de manera secuencial y finalmente se genera una salida (*output*) relacionada con la tarea a resolver.

Para definir las capas una red, vamos a utilizar el paquete ```tf.keras.layers``` cuya documentación está disponible en este [enlace](https://www.tensorflow.org/api_docs/python/tf/keras/layers)

```tf.keras.layers``` permite:
*   Definir la estructura de la red, que tendrá algunos parámetros entrenables (i.e. *weights*).
*   Definir la secuencia de procesado de los datos para obtener una salida (i.e. *forward pass*).

## 1.1 Capa convolucional
Primeramente podemos definir *capas convolucionales 2D* mediante la función ``tensorflow.keras.layers.Conv2D`` [[enlace documentación]](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D), que tiene los principales argumentos:
*   **filters**: número de mapas de salida (i.e. número de convoluciones o *kernels* que aplicamos sobre los datos de entrada).
*   **kernel_size**: tamaño del *kernel* aplicado (tamaño x tamaño)
*   **strides**: desplazamiento de la aplicación del operador de convolución
*   **padding**: tipo de padding applicado
*   **activation**: tipo de función de activación aplicada

Además, cuando se utiliza esta capa como la entrada deberá definir el argumento **input_shape** que indica las dimensiones de los datos a procesar.

A continuación, se muestra un ejemplo:

In [2]:
import tensorflow.keras.layers as layers

# definir primera capa de una red
conv1 = layers.Conv2D(filters=32, 
                       kernel_size=(3, 3), 
                       strides = (1,1),
                       padding = 'same',
                       activation='relu', 
                       input_shape=(32, 32, 3)
                      )
print(conv1)

# definir X capa de una red (observar que no se define la variable "input_shape")
convX = layers.Conv2D(filters=12, 
                       kernel_size=(3, 3), 
                       strides = (1,1),
                       padding = 'same',
                       activation='relu'
                       )
print(convX)

##1.2 Capa Fully Connected
Posteriormente tenemos *capas con conexión completa* (*fully connected*) mediante la función ``tensorflow.keras.layers.Dense``[[enlace documentación]](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), que tiene los principales argumentos:
*   **units**: numero de unidades de salida. 
*   **activation**: función de activación a utilizar
*   **use_bias**: indicador para utilizar sesgo o no

El número de unidades de entrada en esta capa, vendrá definido por la capa anterior conectada en la arquitectura.

A continuación, se muestra un ejemplo:

In [3]:
import tensorflow.keras.layers as layers

fc1 = layers.Dense(units=64, activation='relu', use_bias=True)

print(fc1)

## 1.3 Capa de Pooling Espacial

También exite una etapa dedicada a reducir la dimensionalidad de los datos, cuya nomenclatura es ``tensorflow.keras.layers.MaxPooling2D``[[enlace documentación]](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D)


pool_size: integer or tuple of 2 integers, factors by which to downscale (vertical, horizontal). (2, 2) will halve the input in both spatial dimension. If only one integer is specified, the same window length will be used for both dimensions.
strides: Integer, tuple of 2 integers, or None. Strides values. If None, it will default to pool_size.
padding: 

. Tiene los siguientes argumentos de interés:

*   **pool_size**: tamaño del *kernel* aplicado (tamaño x tamaño)
*   **stride**: desplazamiento de la aplicación del operador de convolución
*   **padding**: tipo de padding applicado

A continuación, se muestra un ejemplo:


In [4]:
import tensorflow.keras.layers as layers

pool1 = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')

print(pool1)

# 2.Definicion de red manual

Una vez estudiadas las capas de la red convolucional, vamos a crear una red utilizando el paquete ```tensorflow.keras.model``` [[enlace documentación]](https://www.tensorflow.org/api_docs/python/tf/keras/Model) definiendo cada una de las capas y su secuenciación.

En esta parte, vamos a tomar como ejemplo la red LENET http://yann.lecun.com/exdb/lenet/ cuya estructura se visualiza a continuacion:

![alt text](http://pytorch.org/tutorials/_images/mnist.png)



Para crear una red, utilizaremos la instruccion ```add``` de la siguiente manera:





In [5]:
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models

# Numero de clases del dataset a analizar
num_classes = 10

# indicamos que definiremos un modelo secuencial de red
model = models.Sequential()

# dimensiones input de mi red
IMG_HEIGHT = 32
IMG_WIDTH = 32
IMG_CHANNELS = 1

# incluir capa convolucional C1
model.add(layers.Conv2D(filters=6, 
                        kernel_size=(3, 3), 
                        strides = (1,1),
                        padding = 'valid',
                        activation='relu', 
                        input_shape=(IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS)))

# incluir average pooling S2
model.add(layers.AveragePooling2D())

# incluir capa convolucional C3
model.add(layers.Conv2D(filters=16, 
                        kernel_size=(3, 3),
                        strides = (1,1),
                        padding = 'valid', 
                        activation='relu'))

# incluir average pooling S4
model.add(layers.AveragePooling2D())

# convertir el volumen de datos en vector fila para conectarlo con capa FC
model.add(layers.Flatten())

# incluir fully convolutional F5
model.add(layers.Dense(units=120, activation='relu'))

# incluir fully convolutional F6
model.add(layers.Dense(units=84, activation='relu'))

# incluir salida OUTPUT
model.add(layers.Dense(units=num_classes, activation = 'softmax'))


Podemos verificar que el modelo se ha creado correctamente y los parámetros existentes con la instrucción ```summary```:



In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 30, 30, 6)         60        
_________________________________________________________________
average_pooling2d (AveragePo (None, 15, 15, 6)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 16)        880       
_________________________________________________________________
average_pooling2d_1 (Average (None, 6, 6, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               69240     
_________________________________________________________________
dense_2 (Dense)              (None, 84)                1

# 3.Cargar una red preentrenada

Si en lugar de definir una red, queremos cargar arquitecturas preexistentes, podemos utilizar el paquete ```tensorflow.keras.applications``` [[enlace documentación]](https://www.tensorflow.org/api_docs/python/tf/keras/applications).

Los argumentos de esta función son:
* **include_top**: si se quiere incluir la última capa fully-connected de la arquitectura
* **weights**: 'None' para inicialización aleatoria o 'imagenet' (pre-entrenados en ImageNet)
* **input_shape**: parámetro opcional que debe ser especificado si include_top=False

No obstante, dependiendo de cada red existen argumentos adicionales que pueden consultarse en el siguiente [enlace](https://keras.io/applications)


In [7]:
import tensorflow.keras.models as models
import tensorflow.keras.applications as applications

# dimensiones de las imagenes a procesar
IMG_SHAPE = (128, 128, 3)

# modelo pre-entrenado VGG16 con tamaño por defecto (224,224,3)
vgg16_model2 = applications.VGG16(include_top=True,
                                weights='imagenet')

vgg16_model2.summary()

# modelo pre-entrenado VGG16 sin incluir capas fc1, fc2 y salida
vgg16_model1 = applications.VGG16(input_shape=IMG_SHAPE, 
                                include_top=False,
                                weights='imagenet')

vgg16_model1.summary()

553467904/553467096 [==============================] - 5s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________